In [32]:
import pandas as pd
import requests
import json
import numpy as np
import datetime

import matplotlib.pyplot as plt
import plotly.express as px

end_date = '23.07.2025'
end_date_to_date = datetime.datetime.strptime(end_date, '%d.%m.%Y') 

response = requests.get(f'https://api.privatbank.ua/p24api/exchange_rates?json&date={end_date}') 
response 

<Response [200]>

In [33]:
tranformed_response = json.loads(response.content)
tranformed_response['exchangeRate']

[{'baseCurrency': 'UAH',
  'currency': 'AUD',
  'saleRateNB': 27.2268,
  'purchaseRateNB': 27.2268},
 {'baseCurrency': 'UAH',
  'currency': 'AZN',
  'saleRateNB': 24.5635,
  'purchaseRateNB': 24.5635},
 {'baseCurrency': 'UAH',
  'currency': 'BYN',
  'saleRateNB': 15.0766,
  'purchaseRateNB': 15.0766},
 {'baseCurrency': 'UAH',
  'currency': 'CAD',
  'saleRateNB': 30.5391,
  'purchaseRateNB': 30.5391},
 {'baseCurrency': 'UAH',
  'currency': 'CHF',
  'saleRateNB': 52.4064,
  'purchaseRateNB': 52.4064,
  'saleRate': 53.45,
  'purchaseRate': 52.35},
 {'baseCurrency': 'UAH',
  'currency': 'CNY',
  'saleRateNB': 5.8209,
  'purchaseRateNB': 5.8209},
 {'baseCurrency': 'UAH',
  'currency': 'CZK',
  'saleRateNB': 1.9846,
  'purchaseRateNB': 1.9846,
  'saleRate': 2.4,
  'purchaseRate': 1.95},
 {'baseCurrency': 'UAH',
  'currency': 'DKK',
  'saleRateNB': 6.546,
  'purchaseRateNB': 6.546},
 {'baseCurrency': 'UAH',
  'currency': 'EUR',
  'saleRateNB': 48.8654,
  'purchaseRateNB': 48.8654,
  'saleRate

In [34]:
currencies = ['USD', 'EUR', 'PLN'] 
currency_df = pd.DataFrame()

def get_exchange_rate(end_date, currencies):
    days_diff = 360
    currency_rates = []

    for i in range(days_diff): 
        current_date = end_date - datetime.timedelta(days=i) 
        response = requests.get(f'https://api.privatbank.ua/p24api/exchange_rates?json&date={current_date.strftime('%d.%m.%Y')}')
        data = json.loads(response.content) 

        for exch_rate in data['exchangeRate']: 
            if exch_rate['currency'] in currencies: 
                currency_rates.append({ 
                    'date': current_date.date(),
                    'currency': exch_rate['currency'],
                    'saleRateNB': exch_rate['saleRateNB'],
                    'purchaseRateNB': exch_rate['purchaseRateNB']
                })
    
    return pd.DataFrame(currency_rates) 

currency_df = get_exchange_rate(end_date_to_date, currencies)
currency_df

,date,currency,saleRateNB,purchaseRateNB
0,2025-07-23,EUR,48.8654,48.8654
1,2025-07-23,PLN,11.4797,11.4797
2,2025-07-23,USD,41.7653,41.7653
3,2025-07-22,EUR,48.7907,48.7907
4,2025-07-22,PLN,11.4986,11.4986
...,...,...,...,...
1069,2024-07-30,PLN,10.3544,10.3544
1070,2024-07-30,USD,41.0478,41.0478
1071,2024-07-29,EUR,44.6324,44.6324
1072,2024-07-29,PLN,10.4437,10.4437


In [35]:
currency_df['date'].nunique()

358

In [36]:
def cumsum_savings(df, daily=2000): 
    savings_df = df.copy() 
    savings_df = savings_df.sort_values(by=['date', 'currency']).reset_index(drop=True)

    savings_df['savings'] = daily / savings_df['saleRateNB'] 
    savings_df['cumulative_savings'] = savings_df.groupby('currency')['savings'].cumsum() 
    return savings_df

currency_df = cumsum_savings(currency_df)
currency_df.sort_values(['currency', 'date'])

,date,currency,saleRateNB,purchaseRateNB,savings,cumulative_savings
0,2024-07-29,EUR,44.6324,44.6324,44.810496,44.810496
3,2024-07-30,EUR,44.4055,44.4055,45.039466,89.849962
6,2024-07-31,EUR,44.4181,44.4181,45.026690,134.876652
9,2024-08-01,EUR,44.4016,44.4016,45.043422,179.920074
12,2024-08-02,EUR,44.4013,44.4013,45.043726,224.963800
...,...,...,...,...,...,...
1061,2025-07-19,USD,41.8670,41.8670,47.770320,17053.484389
1064,2025-07-20,USD,41.8670,41.8670,47.770320,17101.254709
1067,2025-07-21,USD,41.7514,41.7514,47.902585,17149.157294
1070,2025-07-22,USD,41.8194,41.8194,47.824694,17196.981988


In [37]:
fig = px.line(
    currency_df,
    x='date',
    y='cumulative_savings',
    color='currency',
    title='Динаміка заощаджень',
    labels={'cumulative_savings': 'Збереження', 'date': 'Дата', 'currency': 'Валюта'}
)

fig.update_layout(
    title_x=0.5,
    title_font=dict(size=20, color='darkblue'),
    height=500
)

fig.show()

In [38]:
last_day_savings = currency_df[currency_df['date'] == end_date_to_date.date()].copy() 

last_day_savings.loc[:, 'uah_savings'] = last_day_savings['cumulative_savings'] * last_day_savings['saleRateNB']
last_day_savings.style.background_gradient(axis=0, subset=['uah_savings'])

,date,currency,saleRateNB,purchaseRateNB,savings,cumulative_savings,uah_savings
1071,2025-07-23,EUR,48.865400,48.865400,40.928755,15796.394110,771897.116725
1072,2025-07-23,PLN,11.479700,11.479700,174.220581,67316.973156,772778.656743
1073,2025-07-23,USD,41.765300,41.765300,47.886643,17244.868631,720237.111828


Найбільшу суму у гривнях можна відкласти з польського злотого (PLN) - 772 778 грн.

In [39]:
max_uah_savings = last_day_savings['uah_savings'].max() 

last_day_savings['percent_diff'] = ((max_uah_savings - last_day_savings['uah_savings']) / last_day_savings['uah_savings'])
last_day_savings

,date,currency,saleRateNB,purchaseRateNB,savings,cumulative_savings,uah_savings,percent_diff
1071,2025-07-23,EUR,48.8654,48.8654,40.928755,15796.394110,771897.116725,0.001142
1072,2025-07-23,PLN,11.4797,11.4797,174.220581,67316.973156,772778.656743,0.000000
1073,2025-07-23,USD,41.7653,41.7653,47.886643,17244.868631,720237.111828,0.072950


Відсоткової різниці між польським злотим (PLN) та евро (EUR) майже не помітно - 0,11%, що у грошовому еквіваленті становить близько 881 грн.

При цьому різниця між злотим (PLN) та доларем (USD) є більш відчутною - 7,29%. Грошовий еквівалент такої різниці досягає 52 541 грн.

In [40]:
currency_df.to_excel('exchange_rates_usd_eur_pln.xlsx') 